<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#define-location-and-gather-wav-files" data-toc-modified-id="define-location-and-gather-wav-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>define location and gather wav files</a></span></li><li><span><a href="#create-X-categorical-dataframe-and-y-target-dataframe" data-toc-modified-id="create-X-categorical-dataframe-and-y-target-dataframe-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>create X categorical dataframe and y target dataframe</a></span></li><li><span><a href="#define-function-to-extract-feature-data-from-audio" data-toc-modified-id="define-function-to-extract-feature-data-from-audio-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>define function to extract feature data from audio</a></span></li><li><span><a href="#run-functions-and-save-info" data-toc-modified-id="run-functions-and-save-info-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>run functions and save info</a></span></li><li><span><a href="#process-categorical-data" data-toc-modified-id="process-categorical-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>process categorical data</a></span></li><li><span><a href="#save-feature-array-and-target-data-array" data-toc-modified-id="save-feature-array-and-target-data-array-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>save feature array and target data array</a></span></li></ul></div>

In [2]:
# directory library
import os
os.environ['LIBROSA_CACHE_DIR'] = '/tmp/librosa_cache'
import librosa
from pathlib import Path
# progress library
from tqdm import tqdm
# audio modules
import audioread
from librosa import feature
import librosa.display 
import soundfile
import tensorflow as tf
import numpy as np
import pandas as pd 
import scipy
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import re

from collections import Counter

In [3]:
librosa.cache.clear()

# define location and gather wav files

In [4]:
# define path to audio files
audio_path = Path('/Users/badmin/DSC/birdcall_project/trial/')
project = '/Users/badmin/DSC/birdcall_project/'

In [5]:
# gather all wav audio files
files = librosa.util.find_files(audio_path, ext=['wav']) 
filesArray = np.asarray(files)
filesArray

array(['/Users/badmin/DSC/birdcall_project/trial/XC102968.wav.wav',
       '/Users/badmin/DSC/birdcall_project/trial/XC102969.wav.wav',
       '/Users/badmin/DSC/birdcall_project/trial/XC102972.wav.wav', ...,
       '/Users/badmin/DSC/birdcall_project/trial/XC93427.wav.wav',
       '/Users/badmin/DSC/birdcall_project/trial/XC99545.wav.wav',
       '/Users/badmin/DSC/birdcall_project/trial/XC99572.wav.wav'],
      dtype='<U57')

In [6]:

files2df = list(filesArray)
files_df = pd.DataFrame(files2df)

#files_df['filename'] = re.sub("/Users/badmin/DSC/birdcall_project/wav_audio_100/", '', files)

In [7]:
files_df['filename'] = files2df
files_df['filename'].replace(regex=True, inplace=True, to_replace=r'/Users/badmin/DSC/birdcall_project/wav_audio_100/', value=r'')

In [8]:
files_df = files_df['filename']

# create X categorical dataframe and y target dataframe

In [9]:
data = pd.read_csv("data_100.csv")

In [10]:
X_cat_data = data.copy()

In [11]:
X_cat_df =  pd.merge(files_df, X_cat_data, on='filename', how='left')

In [12]:
X_cat_df.drop(columns=['Unnamed: 0', 'filename', 'ebird_code', 'counts'
                ], inplace=True)

In [53]:
X_cat_df.shape

(3200, 3)

In [54]:
# drop unneccessary columns
target_data = data.copy()
target_data.drop(columns=['Unnamed: 0', 'season', 'tod',  'country', 'counts'
                ], inplace=True)

In [55]:
target_df =  pd.merge(files_df, target_data, on='filename', how='left')

In [56]:
y_df = target_df.ebird_code

In [57]:
classes = np.unique(target_df.ebird_code)

In [58]:
classes

array(['amegfi', 'amerob', 'annhum', 'astfly', 'bewwre', 'bkhgro',
       'bnhcow', 'brespa', 'buggna', 'bulori', 'cacwre', 'comgra',
       'daejun', 'dowwoo', 'foxspa', 'gnttow', 'gockin', 'herthr',
       'indbun', 'marwre', 'norcar', 'pasfly', 'purfin', 'rebwoo',
       'scoori', 'sonspa', 'spotow', 'swaspa', 'warvir', 'wesmea',
       'whbnut', 'whcspa'], dtype=object)

# define function to extract feature data from audio

In [32]:
def stat_feats(path):
   
    X, sr = librosa.load(path, sr=32000, duration=30)
    n = len(X)
    n_fft=2048
    y_pad = librosa.util.fix_length(X, n + n_fft // 2, mode='reflect')
    
   
    freqs = np.fft.rfftfreq(n, d=1/sr)
    result = np.array([])
    mean = np.mean(freqs)
    result = np.hstack((result, mean))
    std = np.std(freqs) 
    result = np.hstack((result, std))
    maxv = np.amax(freqs) 
    result = np.hstack((result, maxv))
    minv = np.amin(freqs) 
    result = np.hstack((result, minv))
    median = np.median(freqs)
    result = np.hstack((result, median))
    mode = scipy.stats.mode(freqs)[0][0]
    result = np.hstack((result, mode))

    return result
    

In [15]:
# function defining features to extract to vector
def extract_features(path):
    n_fft = 2048
   
    fmin = 100
    fmax = 14000
    
    #load audio
    X, sr = librosa.load(path, sr=32000, duration=30)
    n = len(X)
    y_pad = librosa.util.fix_length(X, n + n_fft // 2, mode='reflect')
    # extract short term fourier transform sequence information from audio file
    stft = np.abs(librosa.stft(y_pad, 
                               n_fft=n_fft, 
                               center=True,
                               ))

    result = np.array([])

    #spectral mean
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, 
                                                 sr=sr,
                                                 n_fft=n_fft,
                                                 ).T,axis=0)
    result = np.hstack((result, chroma))

    

    #spectral deviation
    chromas = np.std(librosa.feature.chroma_stft(S=stft, 
                                                 sr=sr,
                                                 n_fft=n_fft,
                                                 ).T,axis=0)
    result = np.hstack((result, chromas))

    spec_centroid = np.mean(librosa.feature.spectral_centroid(y=X).T, axis=0)
    result = np.hstack((result, spec_centroid))
  
    spec_centroidM = np.max(librosa.feature.spectral_centroid(y=X).T, axis=0)
    result = np.hstack((result, spec_centroidM))

    # extract mel cepstral coefficients 
    mfccs = np.std(librosa.feature.mfcc(y=X, sr=sr, 
                                        n_fft=n_fft,
                                        lifter=2, 
                                        n_mfcc=12).T, axis=0)
    mfccm = np.mean(librosa.feature.mfcc(y=X, sr=sr, 
                                        n_fft=n_fft,
                                        lifter=2, 
                                        n_mfcc=12).T, axis=0)
    result = np.hstack((result, mfccm))
    mfccM = np.max(librosa.feature.mfcc(y=X, sr=sr, 
                                        n_fft=n_fft,
                                        lifter=2, 
                                        n_mfcc=12).T, axis=0)
    result = np.hstack((result, mfccM))
    mfccme = np.median(librosa.feature.mfcc(y=X, sr=sr, n_fft=n_fft,
                                         lifter=2, n_mfcc=12).T, axis=0)
    result = np.hstack((result, mfccme))
    
    # melspectrogram
    mel = np.mean(librosa.feature.melspectrogram(S=stft, 
                                                 sr=sr,
                                                 n_fft=n_fft,
                                                 n_mels=64).T, axis=0)
 
    result = np.hstack((result, mel))
    mels = np.std(librosa.feature.melspectrogram(S=stft, 
                                                 sr=sr,
                                                 n_fft=n_fft,
                                                 n_mels=64).T, axis=0)
 
    result = np.hstack((result, mels))
    melM = np.max(librosa.feature.melspectrogram(S=stft, 
                                                 sr=sr,
                                                 n_fft=n_fft,
                                                 n_mels=64).T, axis=0)
 
    result = np.hstack((result, melM))
    

# onset envelope

    onset_m = np.max(librosa.onset.onset_strength(S=stft, 
                                             sr=sr,
                                            threshold=0.2).T, axis=0)
    result = np.hstack((result, onset_m))
    onset_M = np.mean(librosa.onset.onset_strength(S=stft, 
                                             sr=sr,
                                            threshold=0.2).T, axis=0)
    result = np.hstack((result, onset_M))

    
    return result



In [58]:
    mfList = []
    for path in tqdm(filesArray):
        y, sr = librosa.load(path, sr=32000, duration=30)
        mfcc = librosa.feature.mfcc(y, sr=sr, n_fft=1024, n_mfcc=12)
        mfList.append(mfcc)


 58%|█████▊    | 1855/3200 [00:34<00:22, 59.03it/s]


100%|██████████| 3200/3200 [00:57<00:00, 55.56it/s]


In [59]:
mfcc_df = pd.DataFrame(mfList)

In [60]:
mfcc_df.to_csv('mfccs100.csv')

In [22]:
def energy(path):
    y, sr = librosa.load(path, sr=32000, duration=30)
    resultS = np.array([])
    
    S = librosa.magphase(librosa.stft(y))[0]
    
    rmss = np.std(librosa.feature.rms(S=S))
    resultS = np.hstack((resultS, rmss))
    rmsM = np.mean(librosa.feature.rms(S=S))
    resultS = np.hstack((resultS, rmsM))

    
    return resultS

# run functions and save info

In [91]:
print(len(filesArray))

6400


In [33]:
# feature extraction
stat_features = []
# extract vector info to X and y is the filenames ordered per extraction
for files in tqdm(filesArray):
    features = stat_feats(files)
    stat_features.append(features)


 40%|███▉      | 1279/3200 [00:18<00:25, 76.45it/s]


 82%|████████▏ | 2637/3200 [00:35<00:06, 92.50it/s]


100%|██████████| 3200/3200 [00:43<00:00, 74.18it/s]


In [81]:
stat_df = pd.DataFrame(stat_features)
stat_df.shape

(3200, 6)

In [82]:
stat_df.to_csv('stat1002.csv')

In [18]:
# feature extraction
Xfeatures = []
# extract vector info to X and y is the filenames ordered per extraction
for files in tqdm(filesArray):
    features = extract_features(files)
    Xfeatures.append(features)


  7%|▋         | 217/3200 [00:49<13:08,  3.78it/s]


 13%|█▎        | 416/3200 [01:36<18:42,  2.48it/s]


 20%|█▉        | 636/3200 [02:22<13:19,  3.21it/s]


 25%|██▌       | 815/3200 [03:09<06:36,  6.01it/s]


 32%|███▏      | 1021/3200 [03:54<13:38,  2.66it/s]


...

 76%|███████▌  | 2428/3200 [09:06<03:18,  3.88it/s]


 83%|████████▎ | 2648/3200 [09:49<02:48,  3.27it/s]


 89%|████████▉ | 2853/3200 [10:33<02:14,  2.58it/s]


 96%|█████████▌| 3059/3200 [11:14<00:50,  2.78it/s]


100%|██████████| 3200/3200 [11:42<00:00,  4.56it/s]


In [19]:
Xfeat_df2 = pd.DataFrame(Xfeatures)
Xfeat_df2.shape

(3200, 256)

In [20]:
Xfeat_df2.to_csv('Xft100_.csv')

In [23]:
# energy extraction
rmsX = []
# extract vector info to X and y is the filenames ordered per extraction
for files in tqdm(filesArray):
    features = energy(files)
    rmsX.append(features)




 14%|█▎        | 433/3200 [00:24<02:50, 16.21it/s]


 27%|██▋       | 858/3200 [00:49<02:15, 17.27it/s]


 39%|███▉      | 1252/3200 [01:12<01:59, 16.25it/s]


 52%|█████▏    | 1658/3200 [01:35<01:18, 19.60it/s]


 66%|██████▌   | 2103/3200 [01:59<01:18, 14.05it/s]


 80%|███████▉  | 2554/3200 [02:23<00:34, 18.73it/s]


 94%|█████████▍| 3014/3200 [02:46<00:10, 17.56it/s]


100%|██████████| 3200/3200 [02:56<00:00, 18.15it/s]


In [24]:
Xrms = pd.DataFrame(rmsX)

In [25]:
Xrms.to_csv('dataRms_.csv')

In [ ]:
result = pd.concat([Xfeat_df2, Xrms], axis=1)

In [41]:
stat_df = pd.read_csv('stat100.csv')

In [42]:
result = pd.concat([result, stat_df], axis=1)

In [43]:
result.shape

(3200, 65)

In [ ]:
result.to_csv('dataFeatures_.csv')

# process categorical data 

In [59]:
cat_data = X_cat_df.copy()
cat_data.head()

,season,tod,country
0,summer,morning,United States
1,summer,morning,United States
2,summer,day,United States
3,summer,morning,United States
4,summer,morning,United States


In [130]:
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [131]:
ohe = OneHotEncoder()
le = LabelEncoder()
lb = LabelBinarizer()

In [62]:
cat_data.drop(columns=['tod'], inplace=True)

In [61]:
cat_data['country'] = pd.Categorical(cat_data['country'])
cat_data['season'] = pd.Categorical(cat_data['season'])

In [134]:
cat = pd.get_dummies(cat_data['country'],prefix='country', drop_first=True)
cat2 = pd.get_dummies(cat_data['season'],prefix='season', drop_first=True)

In [135]:
XFeat_dummies = pd.concat([cat, cat2],axis=1)

In [136]:
XFeat_dummies

,country_Mexico,country_United States,season_spring,season_summer,season_winter
0,0,1,0,1,0
1,0,1,0,1,0
2,0,1,0,1,0
3,0,1,0,1,0
4,0,1,0,1,0
...,...,...,...,...,...
3195,0,1,0,1,0
3196,0,1,0,1,0
3197,0,1,0,0,1
3198,0,1,1,0,0


In [120]:
catEnc = pd.DataFrame()
catEnc['season'] = cat_data.season.cat.codes
catEnc['country'] = cat_data.country.cat.codes

In [121]:
catEnc

,season,country
0,2,2
1,2,2
2,2,2
3,2,2
4,2,2
...,...,...
3195,2,2
3196,2,2
3197,3,2
3198,1,2


# save feature array and target data array

In [74]:
X_df = pd.DataFrame(result)

In [96]:
#XFeat_ohe = pd.concat([X_df, target_ohe], axis=1)
XFeat_enc = pd.concat([X_df, targetEnc], axis=1)

In [99]:
XFeat_enc.to_csv('Xfeat_le.csv')

In [98]:
Xfeat_ohe = pd.concat([X_df, XFeat_dummies], axis=1)

In [100]:
Xfeat_ohe.to_csv('Xfeat_ohe.csv')

In [101]:
df_wT = X_df.copy()

In [102]:
# save y
df_wT['target'] = target_df['ebird_code']

In [103]:
df_wT.to_csv('Xfeat_target.csv')

In [123]:
catEnc.to_csv('catEnc.csv')

In [137]:
XFeat_dummies.to_csv('catOhe.csv')